In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import AlignmentCalculator
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7478689689231248662"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [3]:
video_path = os.path.abspath("./tmp/video/7478689689231248662.mp4")
audio_path = os.path.abspath("./tmp/video/7478689689231248662.wav")

transcript = """
România cedează spațiul aerian aliaților?
Greșit!
Ce se întâmplă de fapt?
Noua lege nu cedează controlul asupra spațiului aerian,
ci stabilește măsuri clare pentru apărarea acestuia
împotriva dronelor și altor amenințări.
Spațiul aerian național este integrat în spațiul aerian al NATO.
Încă de acum 20 de ani, odată cu aderarea României la alianță,
integrarea spațiilor aeriene ale aliaților
este în conformitate cu tratatele la care România este parte.
Degeaba permite intervenția rapidă pentru distrugerea,
neutralizarea și preluarea controlului asupra dronelor
care încalcă spațiul aerian.
Toate deciziile se iau în funcție de nivelul amenințării
și protejarea vieții civile este prioritară.
România nu pierde controlul asupra spațiului său aerian.
Oferă aliaților cadrul legal să ne sprijine
la solicitarea noastră să doborâm dronele.
Nu e haos, nu e pierdere de suveranitate.
România rămâne la butoane și își poate impune oricând restricții.
Siguranță, nu cedare.
Citește legea, informează-te corect și nu pica în capcana dezinformării.
Dacă ai întrebări, lasă-o în comentariu și hai să combatem împreună dezinformarea.
"""

sentiment_results = {
    'positive': 0.06084238737821579,
    'neutral': 0.0,
    'negative': 0.9391575455665588
 }
description = '#MinisterulAparariiNationale #Romania #Securitate #Suveranitate #StopFakeNews #RomaniaPuternica 🇷🇴'

In [4]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)
analysis_result_json

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7478689689231248662.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video addresses the question of whether Romania is ceding its airspace to allies, stating that this is incorrect. It clarifies that a new law establishes measures for defending national airspace against drones and other threats, integrating it with NATO's airspace. The video emphasizes that Romania retains control, can impose restrictions, and that the law provides a legal framework for allies to assist Romania in downing drones when requeste

{'summary': "The video addresses the question of whether Romania is ceding its airspace to allies, stating that this is incorrect. It clarifies that a new law establishes measures for defending national airspace against drones and other threats, integrating it with NATO's airspace. The video emphasizes that Romania retains control, can impose restrictions, and that the law provides a legal framework for allies to assist Romania in downing drones when requested, prioritizing civilian safety. It encourages viewers to read the law and avoid disinformation.",
 'identified_subjects': [{'subject': 'romania', 'stance': 0.8},
  {'subject': 'nato', 'stance': 0.8}],
 'overall_alignment': 0.9}

In [5]:
identified_subjects = analysis_result_json["identified_subjects"]
llm_overall_alignment = analysis_result_json["overall_alignment"]
final_alignment, deterministic_alignment, alignment_conflict = AlignmentCalculator.calculate(identified_subjects, llm_overall_alignment, alpha=0.25)
print(f"llm_overall_alignment: {llm_overall_alignment} \nfinal_alignment: {final_alignment} \ndeterministic_alignment: {deterministic_alignment} \nalignment_conflict: {alignment_conflict}")

llm_overall_alignment: 0.9 
final_alignment: 0.8750000000000001 
deterministic_alignment: 0.8000000000000002 
alignment_conflict: 0.09999999999999987
